In [ ]:
ENV["COLUMNS"] = 1000
ENV["LINES"] = 1000

using ModelingToolkit, DifferentialEquations, Random, Distributions, Plots, CSV, OrdinaryDiffEq, Turing, DataFrames, Symbolics, Serialization
using NaNMath; nm=NaNMath
using StatsPlots; sp=StatsPlots
using ArviZ
using Optim

In [ ]:
# Define parameters
@parameters C_atp_wt C_adp_wt C_cit_wt C_amp_wt C_pep_wt C_pi_wt C_g6p_wt C_g3p_wt C_dhap_wt C_GPI_wt C_PFKM_wt C_AMPK_wt C_ALD_wt # concentrations for wt
@parameters C_atp_zela C_adp_zela C_cit_zela C_amp_zela C_pep_zela C_pi_zela C_g6p_zela C_g3p_zela C_dhap_zela C_GPI_zela C_PFKM_zela C_AMPK_zela C_ALD_zela # Concentrations for zela

@parameters L_PFKM L_PFK2
@parameters Kcat_GPI Kcat_PFKM Kcat_AMPK Kcat_ALD
@parameters Vm_PFK2 Vm_FBP
@parameters Keq_GPI Keq_PFKM Keq_PFK2 Keq_FBP Keq_ALD

@parameters K_GPI_g6p K_GPI_f6p
@parameters Ki_atp Ki_cit Ka_f6p Ka_f16bp Ka_amp Ka_f26bp
@parameters K_f6p K_atp K_f16bp K_adp
@parameters a b v_P
@parameters K2_atp K2_f6p K2_f26bp K2_adp
@parameters K_FBP_f26bp K_FBP_f6p K_pi
@parameters K_ALD_f16bp K_g3p K_dhap
@parameters PP_flux_wt PP_flux_zela

# Define variables
@variables t f6p_wt(t) f16bp_wt(t) f26bp_wt(t) f6p_zela(t) f16bp_zela(t) f26bp_zela(t)
D = Differential(t)

################### Wild type #######################

# Define reaction rates for wild type
r_GPI_wt = C_GPI_wt*(Kcat_GPI/K_GPI_g6p)*(C_g6p_wt-(f6p_wt/Keq_GPI))/
    (1 + C_g6p_wt/K_GPI_g6p + f6p_wt/K_GPI_f6p)

N_PFKM_wt = 1 + L_PFKM*(1 + C_atp_wt/Ki_atp)^4*(1 + C_cit_wt/Ki_cit)^4/
    ((1 + f6p_wt/Ka_f6p + f16bp_wt/Ka_f16bp)^4 * (1 + C_amp_wt/Ka_amp)^4 * (1 + f26bp_wt/Ka_f26bp)^4)

r_PFKM_wt = ((C_PFKM_wt*(Kcat_PFKM/(K_f6p*K_atp))*(C_atp_wt*f6p_wt - C_adp_wt*f16bp_wt/Keq_PFKM))/
    ((1 + f6p_wt/K_f6p)*(1 + C_atp_wt/K_atp) + (1 + f16bp_wt/K_f16bp)*(1 + C_adp_wt/K_adp) -1))*
    (1/N_PFKM_wt)

psi_wt = (Kcat_AMPK*C_AMPK_wt)/(Kcat_AMPK*C_AMPK_wt + v_P)
N_pep_wt = 1 + L_PFK2*(psi_wt/(1-psi_wt))^2
Ki_pep_wt = a + (1 - 1/N_pep_wt)*b
N_PFK2_wt = 1 + (C_pep_wt/Ki_pep_wt)

r_PFK2_wt = (Vm_PFK2/(K2_atp*K2_f6p))*((C_atp_wt*f6p_wt)-(C_adp_wt*f26bp_wt)/Keq_PFK2)/
    ((1+f6p_wt/K2_f6p)*(1+C_atp_wt/K2_atp) + (1+f26bp_wt/K2_f26bp)*(1+C_adp_wt/K2_adp) -1) *
    (1/N_PFK2_wt)

r_FBP_wt = (Vm_FBP/(K_FBP_f26bp))*(f26bp_wt-(f6p_wt*C_pi_wt)/Keq_FBP)/
    ((1+f26bp_wt/K_FBP_f26bp) + (1+f6p_wt/K_FBP_f6p)*(1+C_pi_wt/K_pi)-1)

r_ALD_wt = C_ALD_wt*(Kcat_ALD/K_ALD_f16bp)*(f16bp_wt-(C_g3p_wt*C_dhap_wt)/Keq_ALD)/
    ((1 + f6p_wt/K_ALD_f16bp) + (1 + C_g3p_wt/K_g3p)*(1 + C_dhap_wt/K_dhap) -1)

# Define differential equations for wild type
eqs_wt = [D(f6p_wt) ~ r_GPI_wt - r_PFKM_wt - r_PFK2_wt + r_FBP_wt + PP_flux_wt,
    D(f16bp_wt) ~ r_PFKM_wt - r_ALD_wt,
    D(f26bp_wt) ~ r_PFK2_wt - r_FBP_wt]

@named sys_wt = ODESystem(eqs_wt, t)

# Simplify system for wild type
sys_wt=structural_simplify(sys_wt)


################ Zela #################

# Define reaction rates for zela 
r_GPI_zela = C_GPI_zela*(Kcat_GPI/K_GPI_g6p)*(C_g6p_zela-(f6p_zela/Keq_GPI))/
    (1 + C_g6p_zela/K_GPI_g6p + f6p_zela/K_GPI_f6p)

N_PFKM_zela = 1 + L_PFKM*(1 + C_atp_zela/Ki_atp)^4*(1 + C_cit_zela/Ki_cit)^4/
    ((1 + f6p_zela/Ka_f6p + f16bp_zela/Ka_f16bp)^4 * (1 + C_amp_zela/Ka_amp)^4 * (1 + f26bp_zela/Ka_f26bp)^4)

r_PFKM_zela = ((C_PFKM_zela*(Kcat_PFKM/(K_f6p*K_atp))*(C_atp_zela*f6p_zela - C_adp_zela*f16bp_zela/Keq_PFKM))/
    ((1 + f6p_zela/K_f6p)*(1 + C_atp_zela/K_atp) + (1 + f16bp_zela/K_f16bp)*(1 + C_adp_zela/K_adp) -1))*
    (1/N_PFKM_zela)

psi_zela = (Kcat_AMPK*C_AMPK_zela)/(Kcat_AMPK*C_AMPK_zela + v_P)
N_pep_zela = 1 + L_PFK2*(psi_zela/(1-psi_zela))^2
Ki_pep_zela = a + (1 - 1/N_pep_zela)*b
N_PFK2_zela = 1 + (C_pep_zela/Ki_pep_zela)

r_PFK2_zela = (Vm_PFK2/(K2_atp*K2_f6p))*((C_atp_zela*f6p_zela)-(C_adp_zela*f26bp_zela)/Keq_PFK2)/
    ((1+f6p_zela/K2_f6p)*(1+C_atp_zela/K2_atp) + (1+f26bp_zela/K2_f26bp)*(1+C_adp_zela/K2_adp) -1) *
    (1/N_PFK2_zela)

r_FBP_zela = (Vm_FBP/(K_FBP_f26bp))*(f26bp_zela-(f6p_zela*C_pi_zela)/Keq_FBP)/
    ((1+f26bp_zela/K_FBP_f26bp) + (1+f6p_zela/K_FBP_f6p)*(1+C_pi_zela/K_pi)-1)

r_ALD_zela = C_ALD_zela*(Kcat_ALD/K_ALD_f16bp)*(f16bp_zela-(C_g3p_zela*C_dhap_zela)/Keq_ALD)/
    ((1 + f6p_zela/K_ALD_f16bp) + (1 + C_g3p_zela/K_g3p)*(1 + C_dhap_zela/K_dhap) -1)

# Define differential equations for zela
eqs_zela = [D(f6p_zela) ~ r_GPI_zela - r_PFKM_zela - r_PFK2_zela + r_FBP_zela + PP_flux_zela,
    D(f16bp_zela) ~ r_PFKM_zela - r_ALD_zela,
    D(f26bp_zela) ~ r_PFK2_zela - r_FBP_zela]

@named sys_zela = ODESystem(eqs_zela, t)

# Simplify system for zela
sys_zela=structural_simplify(sys_zela)


In [ ]:
# Shared parameters:
Kcats = [Kcat_GPI => 1.912350597609562e7,
    Kcat_PFKM => 15.5*10^2/(4*10^-5), 
    Kcat_AMPK => 5.732116666666667e-1, 
    Vm_PFK2 => 41.6,
    Vm_FBP => 11.78, 
    Kcat_ALD => 280082.9875518672] 

L = [L_PFKM => 2*10^-1, 
    L_PFK2 => 1*10^-1]
    
Keq = [Keq_GPI => 2.93,
    Keq_PFKM => 480, 
    Keq_PFK2 => 16, 
    Keq_FBP => 56.4458, 
    Keq_ALD => 0.0385] 

K = [K_GPI_g6p => 0.3,
    K_GPI_f6p => 0.123, 
    Ki_atp => 0.1,
    Ki_cit => 0.1, 
    Ka_f6p => 6*10^-2,
    Ka_f16bp => 0.35, 
    Ka_amp => 0.3,
    Ka_f26bp => 5.5*10^-3, 
    K_f6p => 6*10^-2,
    K_atp => 0.15,
    K_f16bp => 0.65,
    K_adp => 0.54, 
    v_P => 5*10^-6,
    a => 0.001,
    b => 3.3, 
    K2_atp => 0.15,
    K2_f6p => 0.032,
    K2_f26bp => 0.008,
    K2_adp => 0.062,
    K_FBP_f26bp => 1*10^-3,
    K_FBP_f6p => 25*10^-3,
    K_pi => 0.23, #
    K_ALD_f16bp => 5*10^-2,
    K_g3p => 0.189,
    K_dhap => 3.5*10^-2] 

p1= [Kcats; L; Keq; K];  

# Strain specific parameters:
u0_wt = [f6p_wt => 1*10^-3,
    f16bp_wt => 1*10^-3,
    f26bp_wt => 1*10^-3]

u0_zela = [f6p_zela => 1*10^-3,
    f16bp_zela => 1*10^-3,
    f26bp_zela => 1*10^-3] 


Cons_wt = [C_atp_wt => exp(-5.647+log(1000)), 
    C_adp_wt => exp(-6.619+log(1000)), 
    C_cit_wt => exp(-3.926+log(1000)),
    C_amp_wt => exp(-12.142+log(1000)),
    C_pep_wt => exp(-12.586+log(1000)),
    C_pi_wt => 1,
    C_g6p_wt => exp(-7.766+log(1000)),
    C_g3p_wt => exp(-9.523+log(1000)),
    C_dhap_wt => exp(-13.055+log(1000)),
    C_GPI_wt => exp(-6.289), 
    C_PFKM_wt => exp(-10.037),
    C_AMPK_wt => exp(-9.764), 
    C_ALD_wt => exp(-5.653),
    PP_flux_wt => 8.8]
    
Cons_zela = [C_atp_zela => exp(-5.676+log(1000)), 
    C_adp_zela => exp(-6.774+log(1000)), 
    C_cit_zela => exp(-3.787+log(1000)),
    C_amp_zela => exp(-11.139+log(1000)),
    C_pep_zela => exp(-12.586+log(1000)),
    C_pi_zela => 1,
    C_g6p_zela => exp(-7.584+log(1000)),
    C_g3p_zela => exp(-8.159), 
    C_dhap_zela => exp(-10.069+log(1000)),
    C_GPI_zela => exp(-5.858), 
    C_PFKM_zela => exp(-9.752),
    C_AMPK_zela => exp(-9.595), 
    C_ALD_zela => exp(-5.322),
    PP_flux_zela => 3.2];


In [ ]:
# Define time span
tspan = (0.0, 1000)
p = [Cons_wt; Cons_zela; p1]
# Define and solve the ODE problem for wild type
prob_wt = ODEProblem(sys_wt, u0_wt, tspan, p, jac = true)
sol_wt = solve(prob_wt, Rodas4(), abstol = 1e-12);
sp.plot(sol_wt)

In [ ]:
# Define and solve the ODE problem for zela
prob_zela = ODEProblem(sys_zela, u0_zela, tspan, p, jac = true)
sol_zela = solve(prob_zela, Rodas4(), abstol = 1e-12);
sp.plot(sol_zela)

In [ ]:
# Calculate rates with defined parameters
r_GPI_para_wt = substitute(r_GPI_wt, p)
r_PFKM_para_wt = substitute(r_PFKM_wt, p)
r_PFK2_para_wt = substitute(r_PFK2_wt, p)
r_FBP_para_wt = substitute(r_FBP_wt, p)
r_ALD_para_wt = substitute(r_ALD_wt, p)
ss_para_wt = [
    f6p_wt => sol_wt[f6p_wt][end],
    f16bp_wt => sol_wt[f16bp_wt][end],
    f26bp_wt => sol_wt[f26bp_wt][end]]

# Calculate derivatives with steady state concentrations:
d_f6p = eval(substitute(r_GPI_para_wt, ss_para_wt) + substitute(r_FBP_para_wt, ss_para_wt) - substitute(r_PFKM_para_wt,ss_para_wt) - substitute(r_PFK2_para_wt, ss_para_wt) + 8.8)
d_f16bp = eval(substitute(r_PFKM_para_wt,ss_para_wt) - substitute(r_ALD_para_wt, ss_para_wt))
d_f26bp = eval(substitute(r_PFK2_para_wt, ss_para_wt) - substitute(r_FBP_para_wt, ss_para_wt))

println("D_f6p= ", d_f6p)
println("D_f16bp= ", d_f16bp)
println("D_f26bp= ", d_f26bp)
println(ss_para_wt)

In [ ]:
# Experimental data for two strains (wild type(wt) and Zela)
g6p_wt = [-7.766+log(1000), 0.319]
g6p_zela = [-7.584+log(1000), 0.331]

dhap_wt = [-13.055+log(1000), 0.165]
dhap_zela = [-10.965+log(1000), 0.161]

g3p_wt = [-9.513+log(1000), 0.113]
g3p_zela = [-10.174+log(1000), 0.117]

atp_wt = [-5.647+log(1000), 0.163]
atp_zela = [-5.676+log(1000), 0.166]

adp_wt = [-6.619+log(1000), 0.167]
adp_zela = [-6.774+log(1000), 0.165]

amp_wt = [-12.142+log(1000), 0.261]
amp_zela = [-11.139+log(1000),  0.257]

pep_wt = [-12.586+log(1000), 0.341]
pep_zela = [-11.229+log(1000), 0.339]

cit_wt = [-3.926+log(1000), 0.219]
cit_zela = [-3.787+log(1000), 0.227]

fdp_wt = [exp(-7.261+log(1000)), 0.281] # fdp = f16bp. The exponential is taken to the mean-value
fdp_zela = [exp(-4.856+log(1000)), 0.239] # fdp = f16bp. The exponential is taken to the mean-value

f6p_wt_val = [exp(-7.760+log(1000)), 0.314] # The exponential is taken to the mean-value
f6p_zela_val = [exp(-7.594+log(1000)), 0.311] # The exponential is taken to the mean-value


# enzyme concentrations
GPI_wt = [-6.289, 0.185]
GPI_zela = [-5.858, 0.103]

PFKM_wt = [-10.037, 0.165]
PFKM_zela = [-9.752, 0.216]

AMPK_wt = [-9.764, 0.087]
AMPK_zela = [-9.595, 0.090]

ALDO_wt = [-5.653, 0.263]
ALDO_zela = [-5.322, 0.079]

data_wt = [atp_wt adp_wt cit_wt amp_wt pep_wt g6p_wt g3p_wt dhap_wt GPI_wt PFKM_wt AMPK_wt ALDO_wt f6p_wt_val fdp_wt]
data_zela = [atp_zela adp_zela cit_zela amp_zela pep_zela g6p_zela g3p_zela dhap_zela GPI_zela PFKM_zela AMPK_zela ALDO_zela f6p_zela_val fdp_zela]

pp_flux_wt = [2*0.0044*1000, 2*8.14*10^-1] 
pp_flux_zela = [2*0.0016*1000, 2*6.22*10^-1] 

PFK_flux_wt = [0.1333*1000,	0.0028*1000]
PFK_flux_zela = [0.0801*1000, 0.0035*1000]

flux_vec = [PFK_flux_wt PFK_flux_zela]

In [ ]:
p_names = [C_atp_wt, C_adp_wt, C_cit_wt, C_amp_wt, C_pep_wt, C_pi_wt, C_g6p_wt, C_g3p_wt, C_dhap_wt, C_GPI_wt, C_PFKM_wt, C_AMPK_wt, C_ALD_wt, PP_flux_wt, C_atp_zela, C_adp_zela, C_cit_zela, C_amp_zela, C_pep_zela, C_pi_zela, C_g6p_zela, C_g3p_zela, C_dhap_zela, C_GPI_zela, C_PFKM_zela, C_AMPK_zela, C_ALD_zela, PP_flux_zela, Kcat_GPI, Kcat_PFKM, Kcat_AMPK, Vm_PFK2, Vm_FBP, Kcat_ALD, L_PFKM, L_PFK2, Keq_GPI, Keq_PFKM, Keq_PFK2, Keq_FBP, Keq_ALD, K_GPI_g6p, K_GPI_f6p, Ki_atp, Ki_cit, Ka_f6p, Ka_f16bp, Ka_amp, Ka_f26bp, K_f6p, K_atp, K_f16bp, K_adp, v_P, a, b, K2_atp, K2_f6p, K2_f26bp, K2_adp, K_FBP_f26bp, K_FBP_f6p, K_pi, K_ALD_f16bp, K_g3p, K_dhap];    

In [ ]:
# Define log likelihood functions
function llik_conc(meas, mean, sd)
    return log(1/(meas*sqrt(2*pi*sd^2))) - (log(meas)-log(mean))^2/(2*sd^2)
end
 
function llik_flux(meas, mean, sd)
    return log(1/(sqrt(2*pi*sd^2))) - (meas-mean)^2/(2*sd^2)
end

In [ ]:
# Define the Bayesian model
@model function mymodelRodas4(data_wt, data_zela, prob_wt, prob_zela, flux, likelihood)
    C_atp_wt ~ LogNormal(data_wt[1,1], data_wt[2,1])
    C_adp_wt ~ LogNormal(data_wt[1,2], data_wt[2,2]) 
    C_cit_wt ~ LogNormal(data_wt[1,3], data_wt[2,3])
    C_amp_wt ~ LogNormal(data_wt[1,4], data_wt[2,4])
    C_pep_wt ~ LogNormal(data_wt[1,5], data_wt[2,5])
    C_pi_wt ~ LogNormal(log(1), 1)
    C_g6p_wt ~ LogNormal(data_wt[1,6], data_wt[2,6]) 
    C_g3p_wt ~ LogNormal(data_wt[1,7], data_wt[2,7]) 
    C_dhap_wt ~ LogNormal(data_wt[1,8], data_wt[2,8])
    C_GPI_wt ~ LogNormal(data_wt[1,9], data_wt[2,9])
    C_PFKM_wt ~ LogNormal(data_wt[1,10], data_wt[2,10])
    C_AMPK_wt ~ LogNormal(data_wt[1,11], data_wt[2,11])
    C_ALD_wt ~ LogNormal(data_wt[1,12], data_wt[2,12])
    PP_flux_wt ~ Normal(8.8, 1.628)
    
    Cons_wt = [C_atp_wt, C_adp_wt, C_cit_wt, C_amp_wt, C_pep_wt, C_pi_wt, C_g6p_wt, C_g3p_wt, C_dhap_wt, C_GPI_wt, C_PFKM_wt, C_AMPK_wt, C_ALD_wt, PP_flux_wt]

    C_atp_zela ~ LogNormal(data_zela[1,1], data_zela[2,1]) 
    C_adp_zela ~ LogNormal(data_zela[1,2], data_zela[2,2]) 
    C_cit_zela ~ LogNormal(data_zela[1,3], data_zela[2,3])
    C_amp_zela ~ LogNormal(data_zela[1,4], data_zela[2,4])
    C_pep_zela ~ LogNormal(data_zela[1,5], data_zela[2,5])
    C_pi_zela ~ LogNormal(log(1), 1)
    C_g6p_zela ~ LogNormal(data_zela[1,6], data_zela[2,6]) 
    C_g3p_zela ~ LogNormal(data_zela[1,7], data_zela[2,7])
    C_dhap_zela ~ LogNormal(data_zela[1,8], data_zela[2,8])
    C_GPI_zela ~ LogNormal(data_zela[1,9], data_zela[2,9])
    C_PFKM_zela ~ LogNormal(data_zela[1,10], data_zela[2,10])
    C_AMPK_zela ~ LogNormal(data_zela[1,11], data_zela[2,11])
    C_ALD_zela ~ LogNormal(data_zela[1,12], data_zela[2,12])
    PP_flux_zela ~ Normal(3.2, 1.244)

    Cons_zela = [C_atp_zela, C_adp_zela, C_cit_zela, C_amp_zela, C_pep_zela, C_pi_zela, C_g6p_zela, C_g3p_zela, C_dhap_zela, C_GPI_zela, C_PFKM_zela, C_AMPK_zela, C_ALD_zela, PP_flux_zela]

    Kcat_GPI ~ LogNormal(log(1.912350597609562e7), 2) 
    Kcat_PFKM ~ LogNormal(log(3.875e7), 2) 
    Kcat_AMPK ~ LogNormal(log(5.73), 2)
    Vm_PFK2 ~ LogNormal(log(41.6), 2) 
    Vm_FBP ~ LogNormal(log(11.78), 2)
    Kcat_ALD ~ LogNormal(log(2.8e5), 2) 

    L_PFKM ~ LogNormal(log(2e-1), 2) 
    L_PFK2 ~ LogNormal(log(1e-1), 2)

    Keq_GPI ~  LogNormal(log(2.92),1.5)
    Keq_PFKM ~ LogNormal(log(480),1.5)
    Keq_PFK2 ~ LogNormal(log(16.0), 1.5)
    Keq_FBP ~ LogNormal(log(56.4458), 1.5)
    Keq_ALD ~ LogNormal(log(0.0385), 1.5) 
    
    K_GPI_g6p ~ LogNormal(log(0.3), 1)
    K_GPI_f6p ~ LogNormal(log(0.123), 1)
    Ki_atp ~ LogNormal(1.26,2) 
    Ki_cit ~ LogNormal(2.98,2) 
    Ka_f6p ~ LogNormal(log(0.43),2) 
    Ka_f16bp ~ LogNormal(log(0.70),2) 
    Ka_amp ~ LogNormal(-5.23,2) 

    Ka_f26bp ~ LogNormal(log(5.5*10^-3), 1)
    K_f6p ~ LogNormal(log(6*10^-2), 1)
    K_atp ~ LogNormal(log(0.15), 1)
    K_f16bp ~ LogNormal(log(0.35), 1)
    K_adp ~ LogNormal(log(0.54), 1)
    v_P ~ LogNormal(log(5*10^-6), 2)
    a ~ LogNormal(log(0.001), 2)
    b ~ LogNormal(log(3.3), 2)
    K2_atp ~ LogNormal(log(0.15), 1)
    K2_f6p ~ LogNormal(log(0.032), 1)
    K2_f26bp ~ LogNormal(log(0.008), 1)
    K2_adp ~ LogNormal(log(0.062), 1)
    K_FBP_f26bp ~ LogNormal(log(1*10^-3), 1)
    K_FBP_f6p ~ LogNormal(log(25*10^-3), 1)
    K_pi ~ LogNormal(log(0.23), 1)
    K_ALD_f16bp ~ LogNormal(log(5*10^-2), 1)
    K_g3p ~ LogNormal(log(0.189), 1)
    K_dhap ~ LogNormal(log(3.5*10^-2), 1)

    p_same = [Kcat_GPI, Kcat_PFKM, Kcat_AMPK, Vm_PFK2, Vm_FBP, Kcat_ALD, L_PFKM, L_PFK2, Keq_GPI, Keq_PFKM, Keq_PFK2, Keq_FBP, Keq_ALD, K_GPI_g6p, K_GPI_f6p, Ki_atp, Ki_cit, Ka_f6p, Ka_f16bp, Ka_amp, Ka_f26bp, K_f6p, K_atp, K_f16bp, K_adp, v_P, a, b, K2_atp, K2_f6p, K2_f26bp, K2_adp, K_FBP_f26bp, K_FBP_f6p, K_pi, K_ALD_f16bp, K_g3p, K_dhap]
    p_total = [Cons_wt; Cons_zela; p_same]
    
    sol_wt = solve(prob_wt, Rodas4(); p=p_total, save_everystep = false, abstol = 1e-12) 
    ss_wt = sol_wt[end]

    sol_zela = solve(prob_zela, Rodas4(); p=p_total, save_everystep = false, abstol = 1e-12) 
    ss_zela = sol_zela[end]

    p_pair = Pair.(p_names, p_total) 
    
    ss_wt_pair = [f6p_wt => ss_wt[1],
        f16bp_wt => ss_wt[2],
        f26bp_wt => ss_wt[3]]

    PFKM_flux_wt = substitute(r_PFKM_wt, [p_pair; ss_wt_pair])
    PFKM_flux_wt_float = PFKM_flux_wt.val

    ss_zela_pair = [f6p_zela => ss_zela[1],
        f16bp_zela => ss_zela[2],
        f26bp_zela => ss_zela[3]]

    PFKM_flux_zela = substitute(r_PFKM_zela, [p_pair; ss_zela_pair])
    PFKM_flux_zela_float = PFKM_flux_zela.val
    
    # If likelihood = 0: Samples from prior distributions
    # If likelihood = 1: Samples from posterior distribtions    
    if likelihood ==1 
        data_wt[1,13] ~ LogNormal(nm.log(ss_wt[1]), data_wt[2,13])
        data_wt[1,14] ~ LogNormal(nm.log(ss_wt[2]), data_wt[2,14])

        data_zela[1,13] ~ LogNormal(nm.log(ss_zela[1]), data_zela[2,13])
        data_zela[1,14] ~ LogNormal(nm.log(ss_zela[2]), data_zela[2,14])

        flux[1,1] ~ Normal(PFKM_flux_wt_float, flux[2,1])
        flux[1,2] ~ Normal(PFKM_flux_zela_float, flux[2,2])
    end
end 

In [ ]:
# Define inital values
inits = []
for x in p
     push!(inits, x.second)
end
init_vec = [inits, inits]

In [ ]:
# Prior sampling
likelihood = 0
model = mymodelRodas4(data_wt, data_zela, prob_wt, prob_zela, flux_vec, likelihood)
sampler = NUTS(1000, 0.95)
samples = 1000
parallel_type = MCMCThreads()
n_chains = 2;

# Uncomment to sample:
#prior = sample(model, sampler, parallel_type, samples, n_chains, initial_params = init_vec)

In [ ]:
# Save samples from prior distribution
#serialize("Model3_prior.jls", prior)

# Load samples from prior distribution
prior = deserialize("Model3_prior.jls");

In [ ]:
# Get a vector with all parameters and their order. Used to check that the order of all vectors containing parameters are in the same order as in the Turing model.
#prior.name_map.parameters

In [ ]:
# Make a dataframe of the sampled prior distributions
prior_df = DataFrame(prior);

In [ ]:
# Solve ODE with each sampled set of parameters
p_prior= select(prior_df, :C_atp_wt, :C_adp_wt, :C_cit_wt, :C_amp_wt, :C_pep_wt, :C_pi_wt, :C_g6p_wt, :C_g3p_wt, :C_dhap_wt, :C_GPI_wt, :C_PFKM_wt, :C_AMPK_wt, :C_ALD_wt, :PP_flux_wt, :C_atp_zela, :C_adp_zela, :C_cit_zela, :C_amp_zela, :C_pep_zela, :C_pi_zela, :C_g6p_zela, :C_g3p_zela, :C_dhap_zela, :C_GPI_zela, :C_PFKM_zela, :C_AMPK_zela, :C_ALD_zela, :PP_flux_zela, :Kcat_GPI, :Kcat_PFKM, :Kcat_AMPK, :Vm_PFK2, :Vm_FBP, :Kcat_ALD, :L_PFKM, :L_PFK2, :Keq_GPI, :Keq_PFKM, :Keq_PFK2, :Keq_FBP, :Keq_ALD, :K_GPI_g6p, :K_GPI_f6p, :Ki_atp, :Ki_cit, :Ka_f6p, :Ka_f16bp, :Ka_amp, :Ka_f26bp, :K_f6p, :K_atp, :K_f16bp, :K_adp, :v_P, :a, :b, :K2_atp, :K2_f6p, :K2_f26bp, :K2_adp, :K_FBP_f26bp, :K_FBP_f6p, :K_pi, :K_ALD_f16bp, :K_g3p, :K_dhap);
tspan = (0,1e3)
ss_prior_wt = zeros(size(p_prior)[1],3)
ss_prior_zela = zeros(size(p_prior)[1],3)
PFKM_flux_zela_vec_prior = []
PFKM_flux_wt_vec_prior = []
p_test = []
i=1
for p_loop in eachrow(Array(p_prior))
    p_test = Pair.(p_names, p_loop)  

    prob_wt_loop = ODEProblem(sys_wt, u0_wt, tspan, p_test, jac = true)
    sol_wt_loop = solve(prob_wt_loop, Rodas4(), abstol = 1e-12);
    ss_prior_wt[i,:] = sol_wt_loop[end]

    prob_zela_loop = ODEProblem(sys_zela, u0_zela, tspan, p_test, jac = true)
    sol_zela_loop = solve(prob_zela_loop, Rodas4(), abstol = 1e-12);
    ss_prior_zela[i,:] = sol_zela_loop[end]

    ss_wt_loop = [f6p_wt => ss_prior_wt[1],
        f16bp_wt => ss_prior_wt[2],
        f26bp_wt => ss_prior_wt[3]]

    ss_zela_loop = [f6p_zela => ss_prior_zela[1],
        f16bp_zela => ss_prior_zela[2],
        f26bp_zela => ss_prior_zela[3]]

    push!(PFKM_flux_wt_vec_prior, substitute(r_PFKM_wt, [p_test; ss_wt_loop]))
    push!(PFKM_flux_zela_vec_prior, substitute(r_PFKM_zela, [p_test; ss_zela_loop]))

    i = i+1
end 

# Convert PFKM flux from num to float
PFKM_flux_wt_vec_prior_float = [i.val for i in PFKM_flux_wt_vec_prior];
PFKM_flux_zela_vec_prior_float = [i.val for i in PFKM_flux_zela_vec_prior];

In [ ]:
# Plot histograms from steady state concentrations and PFKM flux
plt1_wt = histogram(ss_prior_wt[:,1], bin=30, 
        title = "f6p",
        label = "f6p wt", 
        xlabel = "mM",
        fillcolor = :darkcyan,
        legend = false)
    vline!(plt1_wt, [f6p_wt_val[1]], linewidth = 2, linecolor = :magenta3, label = "Measurement")

plt2_wt = histogram(ss_prior_wt[:,2], bin=30, 
        title = "f16bp",
        label = "f16bp wt", 
        xlabel = "mM", 
        fillcolor = :darkcyan,
        legend = false)
    vline!(plt2_wt, [fdp_wt[1]], linewidth = 2, linecolor = :magenta3, label = "Measurement")

plt3_wt = histogram(ss_prior_wt[:,3], bin=30, 
        title = "f26bp",
        label = "f26bp wt", 
        xlabel = "mM", 
        fillcolor = :darkcyan,
        legend = false)

plt4_wt = histogram(PFKM_flux_wt_vec_prior_float, bin=30, 
        title = "PFKM flux",
        label = "SS from prior distribution",
        xlabel = "mM", 
        fillcolor = :darkcyan,
        legend = true)
    vline!(plt4_wt, [flux_vec[1,1]], linewidth = 2, linecolor = :magenta3, label = "Measurement")


plt1_zela = histogram(ss_prior_zela[:,1], bin=30, 
        label = "SS from prior distribution", 
        xlabel = "mM", 
        fillcolor = :darkcyan,
        legend = false)
    vline!(plt1_zela, [f6p_zela_val[1]], linewidth = 2, linecolor = :magenta3, label = "Measurement mean")

plt2_zela = histogram(ss_prior_zela[:,2], bin =30, 
        label = "f16bp zela",
        xlabel = "mM", 
        fillcolor = :darkcyan,
        legend = false)
    vline!(plt2_zela, [fdp_zela[1]], linewidth = 2, linecolor = :magenta3, label = "Measurement mean")

plt3_wt = histogram(ss_prior_zela[:,3], bin=30, 
        title = "f26bp",
        label = "f26bp zela", 
        xlabel = "mM", 
        fillcolor = :darkcyan,
        legend = false)

plt4_zela = histogram(PFKM_flux_zela_vec_prior_float, bin=30, 
        label = "PFKM flux zela",
        xlabel = "mM", 
        fillcolor = :darkcyan,
        legend = false)
    vline!(plt4_zela, [flux_vec[1,2]], linewidth = 2, linecolor = :magenta3, label = "Measurement")
    
plot(plt1_wt, plt2_wt, plt4_wt, plt1_zela, plt2_zela, plt4_zela, layout = (2,3), size = (1000, 600))

In [ ]:
# Make a dataframe of solutions from ODE solving:
df_model3_prior = DataFrame(f6p_wt = ss_prior_wt[:,1], f16bp_wt = ss_prior_wt[:,2], f26bp_wt = ss_prior_wt[:,3], PFKM_flux_wt = PFKM_flux_wt_vec_prior_float, f6p_zela = ss_prior_zela[:,1], f16bp_zela = ss_prior_zela[:,2], f26bp_zela = ss_prior_zela[:,3], PFKM_flux_zela = PFKM_flux_zela_vec_prior_float);

In [ ]:
# Save solutions from ODE solving:
#CSV.write("Model3_ss_dataframe_prior.CSV", df_model3_prior)

# Load solutions from ODE solving:
#df_model3_prior = DataFrame(CSV.File("Model3_ss_dataframe_prior.CSV"))

From here it is posterior

In [ ]:
# Sampling from posterior Distributions
likelihood = 1
model = mymodelRodas4(data_wt, data_zela, prob_wt, prob_zela, flux_vec, likelihood)
sampler = NUTS(1000, 0.95, max_depth=10)
samples = 1000
parallel_type = MCMCThreads()
n_chains = 2;

# Uncomment to sample:
# chain = sample(model, sampler, parallel_type, samples, n_chains, initial_params = init_vec)

In [ ]:
# Save chains from posterior sampling
# serialize("Model3_chains.jls", chain)

# Load chains from posterior sampling
chain = deserialize("Model3_chains.jls")

In [ ]:
# Calculate summarystats and define dataframe from chains
sum_stats = summarystats(chain);
chain_df = DataFrame(chain);
Para = sum_stats[:,2];

In [ ]:
# Minimum and maximum of rhat-values
println(maximum(sum_stats[:,7]))
println(minimum(sum_stats[:,7]))

In [ ]:
# Number of rhat values above 1.1
sum(sum_stats[:,7] .> 1.1)

In [ ]:
# Solve steady states from each samples set of parameters from posterior distribution
p_post= select(chain_df, :C_atp_wt, :C_adp_wt, :C_cit_wt, :C_amp_wt, :C_pep_wt, :C_pi_wt, :C_g6p_wt, :C_g3p_wt, :C_dhap_wt, :C_GPI_wt, :C_PFKM_wt, :C_AMPK_wt, :C_ALD_wt, :PP_flux_wt, :C_atp_zela, :C_adp_zela, :C_cit_zela, :C_amp_zela, :C_pep_zela, :C_pi_zela, :C_g6p_zela, :C_g3p_zela, :C_dhap_zela, :C_GPI_zela, :C_PFKM_zela, :C_AMPK_zela, :C_ALD_zela, :PP_flux_zela, :Kcat_GPI, :Kcat_PFKM, :Kcat_AMPK, :Vm_PFK2, :Vm_FBP, :Kcat_ALD ,:L_PFKM, :L_PFK2, :Keq_GPI, :Keq_PFKM, :Keq_PFK2, :Keq_FBP, :Keq_ALD, :K_GPI_g6p, :K_GPI_f6p, :Ki_atp, :Ki_cit, :Ka_f6p, :Ka_f16bp, :Ka_amp, :Ka_f26bp, :K_f6p, :K_atp, :K_f16bp, :K_adp, :v_P, :a, :b, :K2_atp, :K2_f6p, :K2_f26bp, :K2_adp, :K_FBP_f26bp, :K_FBP_f6p, :K_pi, :K_ALD_f16bp, :K_g3p, :K_dhap);
tspan = (0,1e7)
ss_post_wt = zeros(size(p_post)[1],3)
ss_post_zela = zeros(size(p_post)[1],3)
PFKM_flux_zela_vec_post = []
PFKM_flux_wt_vec_post = []
PFKM_allo_wt_vec = []
PFKM_allo_zela_vec = []
p_test = []
i=1
for p_loop in eachrow(Array(p_post))
    p_test= Pair.(p_names, p_loop)  

    prob_wt_loop = ODEProblem(sys_wt, u0_wt, tspan, p_test, jac = true)
    sol_wt_loop = solve(prob_wt_loop, Rodas4(), abstol = 1e-12);
    ss_post_wt[i,:] = sol_wt_loop[end]

    prob_zela_loop = ODEProblem(sys_zela, u0_zela, tspan, p_test, jac = true)
    sol_zela_loop = solve(prob_zela_loop, Rodas4(), abstol = 1e-12);
    ss_post_zela[i,:] = sol_zela_loop[end]

    ss_wt = [f6p_wt => ss_post_wt[1],
        f16bp_wt => ss_post_wt[2],
        f26bp_wt => ss_post_wt[3]]

    ss_zela = [f6p_zela => ss_post_zela[1],
        f16bp_zela => ss_post_zela[2],
        f26bp_zela => ss_post_zela[3]]
    
    push!(PFKM_flux_wt_vec_post, substitute(r_PFKM_wt, [p_test; ss_wt]))
    push!(PFKM_flux_zela_vec_post, substitute(r_PFKM_zela, [p_test; ss_zela]))

    push!(PFK2_flux_wt_vec_post, substitute(r_PFK2_wt, [p_test; ss_wt]))
    push!(PFK2_flux_zela_vec_post, substitute(r_PFK2_zela, [p_test; ss_zela]))
    
    push!(PFKM_allo_wt_vec, 1/substitute(N_PFKM_zela, [p_test; ss_zela]))
    push!(PFKM_allo_zela_vec, 1/substitute(N_PFKM_zela, [p_test; ss_zela]))

    i = i+1
end 

# Convert from num to float
PFKM_flux_wt_vec_post_float = [i.val for i in PFKM_flux_wt_vec_post];
PFKM_flux_zela_vec_post_float = [i.val for i in PFKM_flux_zela_vec_post];
PFK2_flux_wt_vec_post_float = [i.val for i in PFK2_flux_wt_vec_post];
PFK2_flux_zela_vec_post_float = [i.val for i in PFK2_flux_zela_vec_post];
PFKM_allo_wt_vec_float = [i.val for i in PFKM_allo_wt_vec];
PFKM_allo_zela_vec_float = [i.val for i in PFKM_allo_zela_vec];

In [ ]:
# Define a dataframe of solutions from ODE solving:
df_model3_post = DataFrame(f6p_wt = ss_post_wt[:,1], f16bp_wt = ss_post_wt[:,2], f26bp_wt = ss_post_wt[:,3], PFKM_flux_wt = PFKM_flux_wt_vec_post_float, f6p_zela = ss_post_zela[:,1], f16bp_zela = ss_post_zela[:,2], f26bp_zela = ss_post_zela[:,3], PFKM_flux_zela = PFKM_flux_zela_vec_post_float);

In [ ]:
# Save dataframe of solutions from ODE solving:
# CSV.write("\Model3_ss_dataframe_post.CSV", df_model3_post)

# Load dataframe of solution from ODE solving:
# df_model3_post = DataFrame(CSV.File("Model3_ss_dataframe_post.CSV"))

In [ ]:
# plot of intervals for steady state concentrations
sp.plot([sp.quantile!(ss_post_wt[:,1], 0.025), sp.quantile!(ss_post_wt[:,1], 0.975)], [1,1], label = "f6p", xlabel = "steady state values", xscale=:log10)
sp.plot!([sp.quantile!(ss_post_wt[:,2], 0.025), sp.quantile!(ss_post_wt[:,2], 0.975)], [1.1,1.1], label = "f16bp")
sp.plot!([sp.quantile!(ss_post_wt[:,3], 0.025), sp.quantile!(ss_post_wt[:,3], 0.975)], [0.9,0.9], label = "f26bp")
sp.plot!([sp.quantile!(ss_post_zela[:,1], 0.025), sp.quantile!(ss_post_zela[:,1], 0.975)], [0.99,0.99], label = "f6p zela", xlabel = "steady state values")
sp.plot!([sp.quantile!(ss_post_zela[:,2], 0.025), sp.quantile!(ss_post_zela[:,2], 0.975)], [1.09,1.09], label = "f16bp zela")
sp.plot!([sp.quantile!(ss_post_zela[:,3], 0.025), sp.quantile!(ss_post_zela[:,3], 0.975)], [0.89,0.89], label = "f26bp zela")

In [ ]:
# Plot of steady state flux
sp.plot([sp.quantile!(PFKM_flux_wt_vec_post_float, 0.025), sp.quantile!(PFKM_flux_wt_vec_post_float, 0.975)], [0.9,0.9], label = "PFKM flux wt 95%", legend = :right) # 95 %
sp.plot!([sp.quantile!(PFKM_flux_zela_vec_post_float, 0.025), sp.quantile!(PFKM_flux_zela_vec_post_float, 0.975)], [0.8,0.8], label = "PFKM flux zela 95%") # 95 %
sp.plot!([flux_vec[1,1],flux_vec[1,1]], [0.89, 0.91], label = " wt measurement") # 95 %
sp.plot!([flux_vec[1,2],flux_vec[1,2]], [0.79, 0.81], label = " zela measurement") 

In [ ]:
# Convert to inference data
idata_turing = from_mcmcchains(
    chain;
    prior, 
    library="Turing",
)

In [ ]:
# check of diverging
println("Total # diverging samples: ", sum(idata_turing[:sample_stats][:diverging]))
println("# diverging samples in chain 1: ", sum(idata_turing[:sample_stats][:diverging][:,1]))
println("# diverging samples in chain 2: ", sum(idata_turing[:sample_stats][:diverging][:,2]))

In [ ]:
# Sammarize by Arviz
ArviZ.summarize(idata_turing[:sample_stats], mean, std, rhat)